In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sympy
from numba import jit, njit, prange
from sympy import isprime, primerange
from tqdm import tqdm_notebook as tqdm
import csv
from sklearn.neighbors import KDTree

# np.random.seed(seed=42)
# df = pd.read_csv('1515602.86080513.csv')
# cities = pd.read_csv('input/cities.csv')

In [14]:
a = np.array([0,3,6,9])
b = np.delete(a, 2)
b = np.insert(b, 2, a[2])
#b = np.delete(a, np.where(a==11))
b

array([0, 3, 6, 9])

In [2]:
primes = np.array(list(primerange(0, len(cities))))
is_prime = np.array([0 if not isprime(i) else 1 for i in cities.index]) 
complexes = (cities.X + 1j * cities.Y).values
path = df.Path.values
#path_complexes = np.array([complexes[i] for i in path])
XY = np.array([cities.X.values, cities.Y.values]).T
tree = KDTree(XY, 5)

[primes.shape, complexes.shape, path.shape]

[(17802,), (197769,), (197770,)]

In [26]:
def dist(a, b):
    return np.abs(complexes[a] - complexes[b])

In [27]:
#@jit('i4(i4[:], i4)')
@jit
def closest(path, base_index):
    min_dist = np.inf
    closest_index = -1
    for i in path:
        d = dist(base_index, i)
        if d < min_dist and i != base_index:
            min_dist = d
            closest_index = i
    return closest_index

In [28]:
@jit('i4(i4)')
def closest_prime(base_index):
    min_dist = np.inf
    closest_index = -1
    for i in primes:
        d = dist(base_index, i)
        if d < min_dist and i != base_index:
            min_dist = d
            closest_index = i
    return closest_index

In [4]:
@jit('f8(i8[:], i8, i8)')
def score_between(path, a, b):
    c = np.array(complexes[path[a: b+1]])
    sc = np.sum(np.abs(np.diff(c)))
    pn = 0
    for i in range(a+1, b+1):
        if i % 10 == 0:
            if is_prime[path[i-1]] == False:
                pn += np.abs(complexes[path[i-1]] - complexes[path[i]]) * 0.1
    return sc + pn

In [5]:
score_between(path, 0, len(path)-1)

1515602.8889715623

In [31]:
#@jit('i4[:](i4[:], i4)')
@jit
def closest_two(path, base_index):
    dist_1 = np.inf
    dist_2 = np.inf
    index_1 = -1
    index_2 = -1
    buff = [0]
    for i in path:
        d = dist(base_index, i)
        if d < dist_2 and i != base_index and not i in buff:
            dist_2 = d
            index_2 = i
            buff.append(i)
            if dist_2 < dist_1 and i != base_index:
                dist_1, dist_2 = dist_2, dist_1
                index_1, index_2 = index_2, index_1
    return index_1, index_2           

In [32]:
#@jit('i4[:](i4[:], i4)')
@jit
def closest_three(path, base_index):
    d_list = [np.inf] * 3
    i_list = [-1] * 3
    buff = [0]
    for i in path:
        d = dist(base_index, i)
        if d < d_list[2] and i != base_index and not i in buff:
            d_list[2] = d
            i_list[2] = i
            buff.append(i)
            if d_list[2] < d_list[1]:
                d_list[1:3] = d_list[1:3][::-1]
                i_list[1:3] = i_list[1:3][::-1]
                if d_list[1] < d_list[0]:
                    d_list[0:2] = d_list[0:2][::-1]
                    i_list[0:2] = i_list[0:2][::-1]
    return i_list

In [22]:
def segments(path, path_list):
    copy = path.copy().tolist()
    remove_list = np.concatenate(path_list)
    [copy.remove(i) for i in remove_list]
    del remove_list
    
    for _ in tqdm(range(1, len(path)//20)):
        
        if len(copy) > 3:
            c10, c11, c12 = closest_three(path, path_list[-1][-1])

            copy0 = copy.copy()
            copy1 = copy.copy()
            copy2 = copy.copy()

            if c10 in copy0:
                copy0.remove(c10)
            if c11 in copy1:
                copy1.remove(c11)
            if c12 in copy2:
                copy2.remove(c12)
            
            c20 = closest(copy0, c10)
            c21 = closest(copy1, c11)
            c22 = closest(copy2, c12)

            if c20 in copy0:
                copy0.remove(c20)
            if c21 in copy1:
                copy1.remove(c21)
            if c22 in copy2:
                copy2.remove(c22)

            c30 = closest(copy0, c20)
            c31 = closest(copy1, c21)
            c32 = closest(copy2, c22)

            seg = [[path_list[-1][-1], c10, c20, c30],
                   [path_list[-1][-1], c11, c21, c31],
                   [path_list[-1][-1], c12, c22, c32]]
            sc = [score_between(seg[0], 0, 3),
                  score_between(seg[1], 0, 3),
                  score_between(seg[2], 0, 3)]

            min_index = np.argmin(sc)
            seg_min = seg[min_index][1]
        else:
            seg_min = closest(path, path_list[-1][-1])
            flatten = np.concatenate(path_list)
            if len(flatten) == len(path):
                break

        if seg_min in sum(path_list, []):
            path_list.append([copy[0]])
            copy.remove(copy[0])
        else:
            path_list[-1].append(seg_min)
            copy.remove(seg_min)
            
    return path_list

In [23]:
path_list = [[0]]
path_list = segments(path, path_list)
print(len(path_list), len(sum(path_list, [])))

KeyboardInterrupt: 

In [14]:
# save_list(path_list)
# del path_list
#path_list = load_list()
#path_list

loaded:  39553


In [11]:
#@jit
def segment_list(path, path_list):
    copy = path.copy().tolist()
    remove_list = np.concatenate(path_list)
    [copy.remove(i) for i in remove_list]
    for i in tqdm(range(1, len(path)//10)):
        cl = closest(path, path_list[-1][-1])
        flatten = np.concatenate(path_list)
        if len(flatten) == len(path):
            break
        if cl in flatten:
            path_list.append([copy[0]])
            copy.remove(copy[0])
        else:
            path_list[-1].append(cl)
            copy.remove(cl)
    return path_list

In [60]:
#path_list = [[0]]
path_list = segment_list(path, path_list)
len(sum(path_list, []))

197770

In [12]:
def save_list(path_list):
    with open('path_list2.csv', 'w', newline='') as f:
        writer = csv.writer(f, lineterminator='\n')
        writer.writerows(path_list)
        print('saved: ', len(sum(path_list, [])))

In [61]:
save_list(path_list)

saved:  197770


In [62]:
del path_list

In [13]:
def load_list():
    path_list = []
    with open('path_list2.csv', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            path_list.append([int(i) for i in row])
        print('loaded: ', len(sum(path_list, [])))
        return path_list

In [63]:
path_list = load_list()

loaded:  197770


In [280]:
@jit
def edge_listing(path_list):
    edges = []
    for ls in path_list:
        if len(ls) == 1:
            edges.append(ls[0])
        else:
            edges.append(ls[0])
            edges.append(ls[-1])
    #edges.remove(0)
    return edges

In [313]:
edge_list = edge_listing(path_list)
len(edge_list)

154182

In [364]:
@jit
def join_segments(path_list):
    edge_list = edge_listing(path_list)
    joined_list = path_list[0]
    edge_list.remove(joined_list[0])
    edge_list.remove(edge_list[-1])
    if len(joined_list) > 1:
        edge_list.remove(joined_list[-1])
    
    for i in tqdm(range(len(path_list)-2)):
        cl = closest(np.array(edge_list), joined_list[-1])
        for seg in path_list:
            if cl in seg:
                cl_index = seg.index(cl)
                if len(seg) == 1:
                    edge_list.remove(seg[0])
                else:
                    edge_list.remove(seg[0])
                    edge_list.remove(seg[-1])
                    if cl_index != 0:  
                        seg = seg[::-1]
                joined_list = np.concatenate([joined_list, seg]).tolist()
                break
    joined_list.append(0)
    print(len(joined_list))
    return joined_list

In [365]:
joined = join_segments(path_list)  # 33m 9s

197769


In [372]:
joined = np.array(joined)
score_between(joined, 0, len(joined)-1)

1811006.7213524897

In [373]:
pd.DataFrame({'Path': joined}).to_csv('join_submission.csv', index=False)

In [ ]:
base_list
pairs + singles
groups + pairs + singles

pairs:
    si = path[i]
    s1 = closest_n(path, si)
    s2 = closest_n(path, s1)
    if s1 == s2:
        pairs_list.append(s1)
        pairs_list.append(s2)
    